# Modelos de ML

**Objetivo**: Criar modelos de ML para a projeção de todas as nossas séries.

**Metodologias**: 

- Regressão Linear
- Árvore de decisão
- Random Forest
- XGBoost
- LightGBM

## 0. Setup

In [1]:
%load_ext autotime

time: 110 µs (started: 2024-01-04 12:21:00 -03:00)


In [2]:
#---- Manipulação de dados:

import pandas as pd
import numpy as np

#---- Modelagem:

from hierarchicalforecast.utils import aggregate
from mlforecast import MLForecast
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

#---- Reconciliação

from hierarchicalforecast.methods import BottomUp, TopDown, ERM, OptimalCombination, MinTrace, MiddleOut
from hierarchicalforecast.core import HierarchicalReconciliation

#---- Visualização

import plotly.express as px

time: 1.58 s (started: 2024-01-04 12:21:00 -03:00)


## 1. Dados: vendas de roupas no varejo

In [3]:
dados = pd.read_csv('https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-hierarchical-forecasting/main/retail-usa-clothing.csv')

dados.head()

,date,state,item,quantity,region,country
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic,USA
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic,USA
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic,USA
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic,USA
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic,USA


time: 1 s (started: 2024-01-04 12:21:01 -03:00)


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388024 entries, 0 to 388023
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      388024 non-null  object
 1   state     388024 non-null  object
 2   item      388024 non-null  object
 3   quantity  388024 non-null  int64 
 4   region    388024 non-null  object
 5   country   388024 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.8+ MB
time: 57.1 ms (started: 2024-01-04 12:21:02 -03:00)


## 2. Modificação nos dados 

In [5]:
def clean_data_baseline(df: pd.DataFrame):

    #---- 1. Excluindo a variável de country:

    df = df\
        .drop(columns = 'country')

    #---- 2. Mudando o tipo da variável de date para datetime:

    df['date'] = pd.to_datetime(df['date'])

    #---- 3. Renomeando as variáveis de quantidade de vendas e data:
    # date -> ds
    # quantity -> y

    df = df\
        .rename(columns = {'date': 'ds', 
                           'quantity': 'y'})

    return df

time: 1.98 ms (started: 2024-01-04 12:21:02 -03:00)


In [6]:
df = clean_data_baseline(df = dados)

df.head()

,ds,state,item,y,region
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic


time: 106 ms (started: 2024-01-04 12:21:02 -03:00)


In [7]:
def format_hierarchical_df(df: pd.DataFrame, cols_hierarchical: list):

    #---- 1. Cria uma lista de listas: [[col1], [col1, col2], ..., [col1, col2, coln]]

    hier_list = [cols_hierarchical[:i] for i in range(1, len(cols_hierarchical) + 1)]

    #---- 2. Aplica a função aggregate que formata os dados em que a lib hierarchical pede

    Y_df, S_df, tags = aggregate(df = df, spec = hier_list)

    return Y_df, S_df, tags

time: 1.17 ms (started: 2024-01-04 12:21:03 -03:00)


In [8]:
cols_hierarchical = ['region', 'state', 'item']

Y_df, S_df, tags = format_hierarchical_df(df = df, cols_hierarchical = cols_hierarchical)

time: 689 ms (started: 2024-01-04 12:21:03 -03:00)


In [9]:
display(Y_df.head())
display(Y_df.tail())

,ds,y
unique_id,,
EastNorthCentral,1997-11-25,507
EastNorthCentral,1997-11-26,504
EastNorthCentral,1997-11-27,510
EastNorthCentral,1997-11-28,507
EastNorthCentral,1997-11-29,513


,ds,y
unique_id,,
SouthCentral/Tennessee/womens_shoes,2009-07-24,31
SouthCentral/Tennessee/womens_shoes,2009-07-25,30
SouthCentral/Tennessee/womens_shoes,2009-07-26,31
SouthCentral/Tennessee/womens_shoes,2009-07-27,29
SouthCentral/Tennessee/womens_shoes,2009-07-28,30


time: 9.31 ms (started: 2024-01-04 12:21:03 -03:00)


- **Dados de treino: 25/11/1997 a 31/12/2008**
- **Dados de validação: 01/01/2009 a 28/07/2009**

In [10]:
def split_train_test(df: pd.DataFrame, dt_start_train: str):

    #---- 1. Dados de treino

    train = df.query(f'ds < "{dt_start_train}"')

    #---- 2. Dados de teste:
    
    valid = df.query(f'ds >= "{dt_start_train}"')

    return train, valid

time: 726 µs (started: 2024-01-04 12:21:03 -03:00)


In [11]:
Y_train_df, Y_valid_df = split_train_test(df = Y_df, dt_start_train = '2009-01-01')

time: 84.9 ms (started: 2024-01-04 12:21:03 -03:00)


In [12]:
display(Y_train_df.head())
display(Y_train_df.tail())

,ds,y
unique_id,,
EastNorthCentral,1997-11-25,507
EastNorthCentral,1997-11-26,504
EastNorthCentral,1997-11-27,510
EastNorthCentral,1997-11-28,507
EastNorthCentral,1997-11-29,513


,ds,y
unique_id,,
SouthCentral/Tennessee/womens_shoes,2008-12-27,31
SouthCentral/Tennessee/womens_shoes,2008-12-28,29
SouthCentral/Tennessee/womens_shoes,2008-12-29,28
SouthCentral/Tennessee/womens_shoes,2008-12-30,31
SouthCentral/Tennessee/womens_shoes,2008-12-31,31


time: 11.9 ms (started: 2024-01-04 12:21:03 -03:00)


## 3. Modelagem

In [13]:
#---- Features de data:

from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_7(x):
    return rolling_mean(x, window_size = 7)

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size = 14)

@njit
def rolling_mean_21(x):
    return rolling_mean(x, window_size = 21)

@njit
def rolling_mean_28(x):
    return rolling_mean(x, window_size = 28)

time: 7.44 ms (started: 2024-01-04 12:21:03 -03:00)


In [14]:
def rmse(y_true, y_pred):
    
    return np.sqrt(np.mean(np.square(y_true - y_pred)))

time: 8.57 ms (started: 2024-01-04 12:21:03 -03:00)


In [15]:
n_horizon = Y_valid_df.ds.nunique() # Quantidade de dias para a projeção

time: 5.33 ms (started: 2024-01-04 12:21:03 -03:00)


In [16]:
import optuna
from sklearn.metrics import mean_squared_error

def objective(trial):

    max_depth = trial.suggest_int('max_depth', 3, 15)
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.1, 0.5)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)

    ran_forest = RandomForestRegressor(random_state = 19,
                                       n_estimators = 500,
                                       max_depth = max_depth,
                                       min_samples_split = min_samples_split,
                                       min_samples_leaf = min_samples_leaf,
                                       max_features = max_features
                                      )
    
    models_list = [ran_forest]

    model = MLForecast(models = models_list,
                       freq = 'D',
                       num_threads = 6,
                       lags = [1, 7, 14, 21, 28, 30], 
                       date_features = ['dayofweek', 'month', 'year', 'quarter', 'day', 'week'],
                       lag_transforms = {
                           1: [expanding_mean],
                           7: [rolling_mean_7],
                           14: [rolling_mean_14],
                           21: [rolling_mean_21],
                           28: [rolling_mean_28],
                       }
               )

    model.fit(Y_train_df.reset_index(), id_col = 'unique_id', time_col = 'ds', target_col = 'y', fitted = True)

    Y_hat_df = model.predict(h = n_horizon)

    p = Y_hat_df.reset_index().merge(Y_valid_df.reset_index(), on = ['unique_id', 'ds'], how = 'left')

    error = rmse(p['y'], p['RandomForestRegressor'])
    
    return error

time: 329 ms (started: 2024-01-04 12:21:03 -03:00)


/home/rafa/Documentos/github/multiple-time-series-forecast/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 20)

[I 2024-01-04 12:21:04,227] A new study created in memory with name: no-name-07d23ff7-4b10-4324-9731-0d39f5ab5471
[I 2024-01-04 12:21:27,831] Trial 0 finished with value: 282.43213648843255 and parameters: {'max_depth': 14, 'min_samples_split': 0.9359807433902, 'min_samples_leaf': 0.18883682931614212, 'max_features': 0.8198773317649382}. Best is trial 0 with value: 282.43213648843255.
[I 2024-01-04 12:21:41,378] Trial 1 finished with value: 282.43213648843255 and parameters: {'max_depth': 3, 'min_samples_split': 0.15396266917262183, 'min_samples_leaf': 0.4850521791771094, 'max_features': 0.5686269532876955}. Best is trial 0 with value: 282.43213648843255.
[I 2024-01-04 12:21:54,905] Trial 2 finished with value: 282.43213648843255 and parameters: {'max_depth': 9, 'min_samples_split': 0.8924808875080332, 'min_samples_leaf': 0.2180177715993723, 'max_features': 0.2902915193783454}. Best is trial 0 with value: 282.43213648843255.
[I 2024-01-04 12:24:54,074] Trial 3 finished with value: 214.

time: 1h 3min 21s (started: 2024-01-04 12:21:04 -03:00)


In [18]:
study.best_params

{'max_depth': 7,
 'min_samples_split': 0.10811398161169118,
 'min_samples_leaf': 0.10010742476077619,
 'max_features': 0.9748659806084459}

time: 2.77 ms (started: 2024-01-04 13:24:25 -03:00)


In [25]:
#---- Salvando os melhores parâmetros em um JSON:

import json

with open('random-forest-best-parameters.json', 'w') as jsn:
    json.dump(dict(study.best_params), jsn)

time: 2.16 ms (started: 2024-01-04 13:38:52 -03:00)


In [19]:
study.best_value

197.47421756929234

time: 4.29 ms (started: 2024-01-04 13:24:25 -03:00)
